In [1]:
import numpy as np
from gurobipy import *

In [2]:
with open('UFL.dat') as f:
    dem = np.fromstring(f.readline(), sep=',\t')
    c = np.fromstring(f.readline(), sep=',\t')
    c = c.reshape(len(dem), len(c)//len(dem))
    f = np.fromstring(f.readline(), sep=',\t')

In [3]:
n_clients = dem.shape[0]
print('Demand vector:', dem.shape)
n_facilities = f.shape[0]
print('Fixed-cost vector:', f.shape)
print('Variable-cost matrix:', c.shape)

Demand vector: (45,)
Fixed-cost vector: (40,)
Variable-cost matrix: (45, 40)


In [4]:
# Callback - use lazy constraints to add Benders' cut to master problem by solving the subproblem

def bendersCallback(model, where):
    
    # Found a new (integer) solution to master problem, so use callback
    if where == GRB.callback.MIPSOL:
        
        # Extract 'y' values from master problem solution
        y_bar = model.cbGetSolution([model.getVarByName('y['+str(i)+']') for i in range(n_facilities)])

        # Initialize dual subproblem instance
        sub = Model('subproblem')
        
        # Add dual variables
        u = sub.addVars(n_clients, name='u')
        v = sub.addVars(n_clients, n_facilities, name='v')
        sub.update()
        
        # Add dual constraints
        sub.addConstrs((u[i] - v[i,j] <= c[i,j] for i in range(n_clients) for j in range(n_facilities)), name='dual_constr')
        
        # Initialize objective function
        obj_sub = LinExpr()
        for j in range(n_facilities):
            obj_sub += f[j]*y_bar[j]
        for i in range(n_clients):
            obj_sub += dem[i]*u[i]
            for j in range(n_facilities):
                obj_sub += -1*dem[i]*y_bar[j]*v[i,j]
        
        # Set objective
        sub.setObjective(obj_sub, GRB.MAXIMIZE)
        
        # Gurobi params
        sub.setParam('LogToConsole', 0)
        sub.setParam('InfUnbdInfo', 1)
        sub.setParam('DualReductions', 0)
        
        # Solve dual subproblem
        sub.optimize()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #     BENDERS' FEASIBILITY CUT
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # If subproblem is unbounded
        if sub.status == 5:

            # Extract Unbounded Rays 'u' & 'v' values from solution of subproblem
            u_bar = np.zeros(n_clients)
            for i in range(n_clients):
                u_bar[i] = u[i].UnbdRay

            v_bar = np.zeros((n_clients, n_facilities))
            for i in range(n_clients):
                for j in range(n_facilities):
                    v_bar[i,j] = v[i,j].UnbdRay

            # Set-up cut
            expr = LinExpr()
            for i in range(n_clients):
                expr += dem[i]*u_bar[i]
                for j in range(n_facilities):
                    expr += -1*dem[i]*v_bar[i,j]*y[j]
            
            # Add Benders' feasibility cut to master problem
            master.cbLazy(expr <= 0)

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #     BENDERS' OPTIMALITY CUT
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # If not unbounded, then subproblem is feasible
        else:

            # Extract Extreme Points 'u' & 'v' values from solution of subproblem
            u_bar = np.zeros(n_clients)
            for i in range(n_clients):
                u_bar[i] = u[i].X

            v_bar = np.zeros((n_clients, n_facilities))
            for i in range(n_clients):
                for j in range(n_facilities):
                    v_bar[i,j] = v[i,j].X

            # Set-up cut
            expr = LinExpr()
            for j in range(n_facilities):
                expr += f[j]*y[j]
            for i in range(n_clients):
                expr += dem[i]*u_bar[i]
                for j in range(n_facilities):
                    expr += -1*dem[i]*v_bar[i,j]*y[j]

            # Add Benders' optimality cut to master problem
            master.cbLazy(z >= expr)

In [5]:
# Initialize master problem instance
master = Model('master_problem')

# Add binary variables
z = master.addVar(obj=1, name='z')
y = master.addVars(n_facilities, obj=0, vtype=GRB.BINARY, name='y')
master.update()

# Set objective
master.ModelSense=GRB.MINIMIZE

# Gurobi params
master.setParam('LazyConstraints', 1)

# Solve master problem with callback solving the subproblem
master.optimize(bendersCallback)

Academic license - for non-commercial use only
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 0 rows, 41 columns and 0 nonzeros
Variable types: 1 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 41 columns, 0 nonzeros
Variable types: 1 continuous, 40 integer (40 binary)

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 156089.564    0    1          - 156089.564      -     -    1s
H    0     0                    202452.00000 156089.564  22.9%     -    2s
H    0     0                    190628.00000 156089.564  18.1%     -    3s
     0     0 164570.738    0    6 1

In [10]:
#######################
#  DISABLE PRE-SOLVE  #
#  DISABLE HEURISTICS #
#    DISABLE CUTS     #
#######################

# Initialize master problem instance
master = Model('master_problem')

# Add binary variables
z = master.addVar(obj=1, name='z')
y = master.addVars(n_facilities, obj=0, vtype=GRB.BINARY, name='y')
master.update()

# Set objective
master.ModelSense=GRB.MINIMIZE

# Gurobi params
master.setParam('Presolve', 0)
master.setParam('Heuristics', 0)
master.setParam('Cuts', 0)
master.setParam('LazyConstraints', 1)

# Solve master problem with callback solving the subproblem
master.optimize(bendersCallback)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 0 rows, 41 columns and 0 nonzeros
Variable types: 1 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Variable types: 1 continuous, 40 integer (40 binary)

Root relaxation: objective 0.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 156089.564    0    1          - 156089.564      -     -    1s
     0     0 156089.564    0    1      

In [6]:
# Write lp file
master.write('benders_lazy.lp')